In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from pprint import pprint
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
%matplotlib inline
# %matplotlib notebook
from Ipython import display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [2]:
robot1 = pd.read_csv('RobotDataset/Robot1')
robot2 = pd.read_csv('RobotDataset/Robot2')

In [9]:
display(robot1.head())
display(robot2.head())
display(robot1.describe())
display(robot2.describe())

,1 1 1 1 1 3 1 data_5
0,1 1 1 1 1 3 2 data_6
1,1 1 1 1 3 2 1 data_19
2,1 1 1 1 3 3 2 data_22
3,1 1 1 2 1 2 1 data_27
4,1 1 1 2 1 2 2 data_28


,1 1 1 1 1 1 2 data_2
0,1 1 1 1 1 2 1 data_3
1,1 1 1 1 1 2 2 data_4
2,0 1 1 1 1 3 1 data_5
3,0 1 1 1 1 4 1 data_7
4,1 1 1 1 2 1 1 data_9


,1 1 1 1 1 3 1 data_5
count,123
unique,123
top,0 1 3 2 2 4 2 data_136
freq,1


,1 1 1 1 1 1 2 data_2
count,121
unique,121
top,0 1 3 2 2 4 2 data_136
freq,1


In [12]:
robot1.isnull().values.any()
robot2.isnull().values.any()

False

In [36]:
from heapq import heappush, heappop

In [39]:
class KNN:
    def fit(self, train_X, train_y, k=3):
        self.train_X = train_X
        self.train_y = train_y
        self.k = k
        
    def euclid(test_row, train_row):
        dis = np.sqrt(np.sum([(x-y)**2 for x,y in zip(test_row, train_row)]))
        return dis
    
    def predict_row(self, test_row, calulate_distance=euclid):
        heap = []
        for (index, train_row) in train_X.iterrows():
            dis = self.calulate_distance(train_row, test_row)
            heappush(heap, (dis, train_y.iloc[i,0]))
            if len(heap) > self.k:
                heappop(heap) 
        elem, count = np.unique([i for j,i in heap],return_counts=True)
        return elem[argmax(count)]
    
    def predict(self, test_X):
        test_y = pd.DataFrame()
        test_y['predicted'] = [predict_row(row) for i, row in test_X.iterrows()]
        return test_y
    
    def evalulate_result(self, )